In [1]:
import cProfile
import os
import pickle
import pstats
import re
import time
from collections import namedtuple, defaultdict
from typing import List
from multiprocessing import Pool
import networkx as NX
import pandas as pds
import matplotlib.pyplot as plt
import itertools
import math
import re
import statistics
from pprint import pprint

import rich.progress
import scipy
from scipy import stats as scistats
from sklearn import linear_model, preprocessing
from tabulate import tabulate
from pathlib import Path
import pickle
import random
import argparse
import sys

import data_types
from db_actions import db_actions
import numpy as np
from my_yaml import yaml_load, yaml_dump
from alive_progress import alive_bar
import seaborn as sns
from data_types import PickleOut
import commons
import rich.status
from rich.pretty import pprint
from rich import inspect
import textwrap

# Full Models

In [2]:
rc = commons.rc

In [179]:
pds.set_option("display.float_format", lambda x: f"{x:.4f}" if -10<=x<=10 else f"{x:.3e}")

In [28]:
namesLUT = {
    'Linear Regression': "Ordinary Least Squares", 'SVR - rbf': "SVR", 'Theil Sen':"Theil Sen", 'Ridge': "Ridge",
      'Huber Regressor': "Huber Regressor", 'Ridge CV':"Ridge", 'SVR - poly':"SVR", 'SVR - linear':"SVR",
      'Lasso CV':"Lasso", 'Stochastic Gradient Descent':"Stochastic Gradient Descent", 'Elastic Net CV':"Elastic Net",
      'Bayesian Ridge':"Bayesian Ridge", 'Automatic Relevance Determination Regression':"Automatic Relevance Determination",
      'Passive Aggressive Regressor':"Passive Aggressive", 'Lasso':"Lasso", 'SVR - sigmoid':"SVR",
     'Elastic Net':"Elastic Net"
}

In [218]:
fmDir = Path("models")
col = list()
for f in os.listdir(fmDir):
    if f == "bestModel.pickle":
        continue
    if not re.match(".*\.pickle$", f):
        continue
    fP = fmDir.joinpath(f)
    # rc.log(fP)
    with open(fP, "br") as f2:
        fmP:PickleOut = pickle.load(f2)
        longname, regr, test_confidence, polyDeg, full_confidence, unknown_confidence, train_confidence, bonusPickleInfo = fmP
        # rc.log(fmP)
        col.append([longname, None, test_confidence, polyDeg, full_confidence, unknown_confidence, train_confidence, None])
t = pds.DataFrame(data=col, columns=PickleOut._fields)

In [219]:
t = t.drop(["regr", "bonusPickleInfo"],axis=1)

In [220]:
t["conf"] = t["full_confidence"]
t  =t.drop(["full_confidence", "train_confidence", "test_confidence", "unknown_confidence"],axis=1)

In [221]:
t = t.rename(columns={"longname":"name","polyDeg": "deg"})

In [222]:
col = list()
for i, d in t.groupby(["deg", "name"]):
    maxi = d.conf.max()
    col.append([i[0],i[1], maxi])
t = pds.DataFrame(data=col, columns=["deg", "name", "conf"])

In [223]:
t["conf"] = [round(c,4) for c in t["conf"]]

In [224]:
t = t.sort_values(["conf"], ascending=False)

In [225]:
t.name.unique()

array(['Linear Regression', 'SVR - rbf', 'Theil Sen', 'Ridge',
│      'Huber Regressor', 'Ridge CV', 'SVR - poly', 'SVR - linear',
│      'Lasso CV', 'Stochastic Gradient Descent', 'Elastic Net CV',
│      'Bayesian Ridge', 'Automatic Relevance Determination Regression',
│      'Passive Aggressive Regressor', 'Lasso', 'SVR - sigmoid',
│      'Elastic Net'], dtype=object)

In [226]:
rc.print(t.to_latex(index=False))

\begin{tabular}{rlr}
\toprule
 deg &                                         name &       conf \\
\midrule
   5 &                            Linear Regression &     1.0000 \\
   4 &                            Linear Regression &     0.9961 \\
   4 &                                    SVR - rbf &     0.9749 \\
   3 &                            Linear Regression &     0.9394 \\
   3 &                                    Theil Sen &     0.9392 \\
   2 &                                    SVR - rbf &     0.9348 \\
   1 &                                    SVR - rbf &     0.9325 \\
   3 &                                        Ridge &     0.9232 \\
   3 &                              Huber Regressor &     0.9201 \\
   3 &                                    SVR - rbf &     0.9131 \\
   3 &                                     Ridge CV &     0.9063 \\
   1 &                                   SVR - poly &     0.8976 \\
   3 &                                 SVR - linear &     0.8950 \\
   2 &                                   SVR - poly &     0.8760 \\
   3 &                                     Lasso CV &     0.8711 \\
   2 &                            Linear Regression &     0.8699 \\
   3 &                  Stochastic Gradient Descent &     0.8695 \\
   2 &                              Huber Regressor &     0.8688 \\
   3 &                               Elastic Net CV &     0.8672 \\
   2 &                                        Ridge &     0.8667 \\
   2 &                               Bayesian Ridge &     0.8620 \\
   2 & Automatic Relevance Determination Regression &     0.8602 \\
   2 &                                     Ridge CV &     0.8572 \\
   3 &                 Passive Aggressive Regressor &     0.8428 \\
   2 &                               Elastic Net CV &     0.8427 \\
   2 &                                     Lasso CV &     0.8425 \\
   2 &                  Stochastic Gradient Descent &     0.8416 \\
   1 &                            Linear Regression &     0.8338 \\
   1 &                                     Ridge CV &     0.8294 \\
   1 & Automatic Relevance Determination Regression &     0.8282 \\
   2 &                                 SVR - linear &     0.8278 \\
   1 &                              Huber Regressor &     0.8272 \\
   1 &                               Bayesian Ridge &     0.8272 \\
   2 &                 Passive Aggressive Regressor &     0.8253 \\
   1 &                                     Lasso CV &     0.8232 \\
   1 &                                        Ridge &     0.8190 \\
   2 &                                    Theil Sen &     0.8168 \\
   2 &                                        Lasso &     0.8166 \\
   1 &                               Elastic Net CV &     0.8160 \\
   1 &                                 SVR - linear &     0.8136 \\
   1 &                                SVR - sigmoid &     0.8111 \\
   3 &                                   SVR - poly &     0.8070 \\
   4 &                                   SVR - poly &     0.7986 \\
   1 &                                    Theil Sen &     0.7923 \\
   1 &                  Stochastic Gradient Descent &     0.7809 \\
   3 &                                  Elastic Net &     0.7803 \\
   1 &                 Passive Aggressive Regressor &     0.7781 \\
   1 &                                        Lasso &     0.7699 \\
   4 &                                 SVR - linear &     0.6880 \\
   1 &                                  Elastic Net &     0.6184 \\
   2 &                                  Elastic Net &     0.5736 \\
   3 &                                        Lasso &     0.3124 \\
   2 &                                SVR - sigmoid &     0.1548 \\
   4 &                 Passive Aggressive Regressor &     0.0412 \\
   3 &                                SVR - sigmoid &    -0.0004 \\
   4 &                                SVR - sigmoid & -2.556e+03 \\
\bottomrule
\end{tabular}

In [227]:
rc.print(t.sort_values(["deg","conf"], ascending=(True, False)).to_latex(index=False))

\begin{tabular}{rlr}
\toprule
 deg &                                         name &       conf \\
\midrule
   1 &                                    SVR - rbf &     0.9325 \\
   1 &                                   SVR - poly &     0.8976 \\
   1 &                            Linear Regression &     0.8338 \\
   1 &                                     Ridge CV &     0.8294 \\
   1 & Automatic Relevance Determination Regression &     0.8282 \\
   1 &                              Huber Regressor &     0.8272 \\
   1 &                               Bayesian Ridge &     0.8272 \\
   1 &                                     Lasso CV &     0.8232 \\
   1 &                                        Ridge &     0.8190 \\
   1 &                               Elastic Net CV &     0.8160 \\
   1 &                                 SVR - linear &     0.8136 \\
   1 &                                SVR - sigmoid &     0.8111 \\
   1 &                                    Theil Sen &     0.7923 \\
   1 &                  Stochastic Gradient Descent &     0.7809 \\
   1 &                 Passive Aggressive Regressor &     0.7781 \\
   1 &                                        Lasso &     0.7699 \\
   1 &                                  Elastic Net &     0.6184 \\
   2 &                                    SVR - rbf &     0.9348 \\
   2 &                                   SVR - poly &     0.8760 \\
   2 &                            Linear Regression &     0.8699 \\
   2 &                              Huber Regressor &     0.8688 \\
   2 &                                        Ridge &     0.8667 \\
   2 &                               Bayesian Ridge &     0.8620 \\
   2 & Automatic Relevance Determination Regression &     0.8602 \\
   2 &                                     Ridge CV &     0.8572 \\
   2 &                               Elastic Net CV &     0.8427 \\
   2 &                                     Lasso CV &     0.8425 \\
   2 &                  Stochastic Gradient Descent &     0.8416 \\
   2 &                                 SVR - linear &     0.8278 \\
   2 &                 Passive Aggressive Regressor &     0.8253 \\
   2 &                                    Theil Sen &     0.8168 \\
   2 &                                        Lasso &     0.8166 \\
   2 &                                  Elastic Net &     0.5736 \\
   2 &                                SVR - sigmoid &     0.1548 \\
   3 &                            Linear Regression &     0.9394 \\
   3 &                                    Theil Sen &     0.9392 \\
   3 &                                        Ridge &     0.9232 \\
   3 &                              Huber Regressor &     0.9201 \\
   3 &                                    SVR - rbf &     0.9131 \\
   3 &                                     Ridge CV &     0.9063 \\
   3 &                                 SVR - linear &     0.8950 \\
   3 &                                     Lasso CV &     0.8711 \\
   3 &                  Stochastic Gradient Descent &     0.8695 \\
   3 &                               Elastic Net CV &     0.8672 \\
   3 &                 Passive Aggressive Regressor &     0.8428 \\
   3 &                                   SVR - poly &     0.8070 \\
   3 &                                  Elastic Net &     0.7803 \\
   3 &                                        Lasso &     0.3124 \\
   3 &                                SVR - sigmoid &    -0.0004 \\
   4 &                            Linear Regression &     0.9961 \\
   4 &                                    SVR - rbf &     0.9749 \\
   4 &                                   SVR - poly &     0.7986 \\
   4 &                                 SVR - linear &     0.6880 \\
   4 &                 Passive Aggressive Regressor &     0.0412 \\
   4 &                                SVR - sigmoid & -2.556e+03 \\
   5 &                            Linear Regression &     1.0000 \\
\bottomrule
\end{tabular}

In [210]:
fmDir = Path("cvmodels")
col = list()
for f in os.listdir(fmDir):
    if f == "bestModel.pickle":
        continue
    if not re.match(".*\.pickle$", f):
        continue
    m = re.match(".*\..*_CV-(.*)_U-(.*)\.pickle",f)
    cv, uk = m.group(1,2)
    fP = fmDir.joinpath(f)
    # rc.log(fP)
    with open(fP, "br") as f2:
        fmP:PickleOut = pickle.load(f2)
        longname, regr, test_confidence, polyDeg, full_confidence, unknown_confidence, train_confidence,_ = fmP
        # rc.log(fmP)
        col.append([longname, test_confidence, polyDeg, full_confidence, unknown_confidence, train_confidence, commons.jamGeomean([test_confidence, full_confidence, train_confidence]), cv, uk])
t = pds.DataFrame(data=col, columns=["name","test_conf", "deg", "full_conf","unknown_conf","train_conf","gconf", "cvSplit", "ukSplit"])

In [211]:
fmDir = Path("cvmodels")

wfNames=["eager","methylseq","viralrecon","chipseq","sarek"]
models = [
    'Linear', 'SVR-rbf','TheilSen',
      'Huber', 'RidgeCV','SVR-poly', 'SVR-linear',
      'LassoCV', 'SGD', 'ElasticNetCV',
      'BayesianRidge', 'ARD',
      'PA', 'Lasso', 'SVR-sigmoid',
     'ElasticNet','Ridge'
]

c = dict()

for f in os.listdir(fmDir):
    if f == "bestModel.pickle":
        continue
    for m in models:
        if m in f:
            c[m] = c.get(m,0)+1
            break
pprint(c)

{
│   'SVR-rbf': 60,
│   'ElasticNet': 60,
│   'ARD': 40,
│   'Huber': 40,
│   'Ridge': 60,
│   'SVR-linear': 60,
│   'SGD': 60,
│   'PA': 60,
│   'Linear': 60,
│   'ElasticNetCV': 60,
│   'TheilSen': 60,
│   'SVR-sigmoid': 60,
│   'LassoCV': 60,
│   'Lasso': 60,
│   'SVR-poly': 60,
│   'RidgeCV': 60,
│   'BayesianRidge': 40
}

In [212]:
t

,name,test_conf,deg,full_conf,unknown_conf,train_conf,gconf,cvSplit,ukSplit
0,SVR - rbf,0.2394,3,0.2885,0.2372,0.3520,0.2925,eager,sarek
1,Elastic Net,0.5675,1,0.6339,0.6477,0.6574,0.6192,chipseq,eager
2,Automatic Relevance Determination Regression,0.7705,1,0.8242,0.8061,0.8408,0.8116,methylseq,chipseq
3,Huber Regressor,-0.2161,2,0.6876,0.6364,0.8883,0.3787,methylseq,chipseq
4,Ridge,0.8401,1,0.8118,0.7861,0.8175,0.8231,eager,chipseq
...,...,...,...,...,...,...,...,...,...
955,Elastic Net,0.7709,3,0.7911,0.7277,0.8611,0.8073,viralrecon,chipseq
956,Bayesian Ridge,0.7939,1,0.8200,0.8495,0.8252,0.8130,chipseq,eager
957,Ridge CV,0.7871,3,0.8122,0.7758,0.8251,0.8080,eager,chipseq
958,SVR - linear,-0.0843,3,0.4941,0.0643,0.7010,0.3283,sarek,chipseq


In [213]:
t = t.join(t.groupby(["deg", "name"])["gconf"].aggregate(commons.jamGeomean),on=["deg", "name"],rsuffix="avg")\
.join(t.groupby(["deg", "name"])["unknown_conf"].aggregate(commons.jamGeomean),on=["deg", "name"],rsuffix="avg").sort_values(["deg","name", "gconfavg"]).reset_index(drop=True)

In [214]:
t["name"] = t["name"].apply(lambda x: namesLUT[x])

In [215]:
#rc.print(
out = t.join(t.groupby(["deg", "cvSplit", "ukSplit", "name"])["gconfavg"].aggregate(max),on=["deg", "cvSplit", "ukSplit", "name"],rsuffix="_maxi")\
.query("gconfavg==gconfavg_maxi").sort_values(["deg", "gconfavg"],  ascending=(True, False)).reset_index(drop=True)[["deg","name","test_conf","full_conf","train_conf", "unknown_conf", "unknown_confavg","gconf", "gconfavg","ukSplit","cvSplit"]]

In [216]:
out

,deg,name,test_conf,full_conf,train_conf,unknown_conf,unknown_confavg,gconf,gconfavg,ukSplit,cvSplit
0,1,SVR,0.7779,0.8577,0.9175,0.6472,0.7343,0.8501,0.8388,eager,chipseq
1,1,SVR,0.7259,0.8786,0.9132,0.7711,0.7343,0.8374,0.8388,eager,methylseq
2,1,SVR,0.7118,0.8592,0.9010,0.8204,0.7343,0.8222,0.8388,chipseq,methylseq
3,1,SVR,0.7590,0.8278,0.9078,0.7444,0.7343,0.8305,0.8388,viralrecon,sarek
4,1,SVR,0.7116,0.8165,0.9426,0.7202,0.7343,0.8211,0.8388,viralrecon,methylseq
...,...,...,...,...,...,...,...,...,...,...,...
595,3,Ordinary Least Squares,-2.892e+21,-6.972e+20,0.9534,-8.080e+20,-1.706e+16,-1.011e+14,-6.131e+10,chipseq,methylseq
596,3,Ordinary Least Squares,-8.463e+04,-6.665e+05,1.0000,-8.526e+06,-1.706e+16,-3.043e+03,-6.131e+10,sarek,viralrecon
597,3,Ordinary Least Squares,-1.375e+19,-4.356e+20,0.9428,-2.497e+23,-1.706e+16,-1.456e+13,-6.131e+10,sarek,eager
598,3,Ordinary Least Squares,-5.146e+20,-2.153e+20,0.9525,-1.162e+21,-1.706e+16,-3.843e+13,-6.131e+10,eager,chipseq


In [217]:
rc.print(out.groupby(["deg","name"]).aggregate(lambda x: x.iloc[0]).reset_index().sort_values(["deg","gconfavg"], ascending=(True, False))[["deg","name","gconfavg","unknown_confavg"]].to_latex(index=False))

\begin{tabular}{rlrr}
\toprule
 deg &                              name &   gconfavg &  unknown\_confavg \\
\midrule
   1 &                               SVR &     0.8388 &           0.7343 \\
   1 &            Ordinary Least Squares &     0.8234 &           0.8050 \\
   1 & Automatic Relevance Determination &     0.8191 &           0.8118 \\
   1 &                             Ridge &     0.8179 &           0.8002 \\
   1 &                   Huber Regressor &     0.8167 &           0.8040 \\
   1 &                    Bayesian Ridge &     0.8161 &           0.8038 \\
   1 &                             Lasso &     0.8108 &           0.7920 \\
   1 &                       Elastic Net &     0.8045 &           0.7887 \\
   1 &       Stochastic Gradient Descent &     0.7856 &           0.7648 \\
   1 &                         Theil Sen &     0.7815 &           0.7553 \\
   1 &                Passive Aggressive &     0.7686 &           0.6615 \\
   2 &                             Lasso &     0.8150 &           0.6157 \\
   2 &       Stochastic Gradient Descent &     0.8149 &           0.6499 \\
   2 &                       Elastic Net &     0.8144 &           0.6825 \\
   2 &                             Ridge &     0.8066 &           0.4598 \\
   2 &                    Bayesian Ridge &     0.7714 &           0.3125 \\
   2 &                               SVR &     0.7318 &           0.6373 \\
   2 & Automatic Relevance Determination &     0.7158 &           0.4793 \\
   2 &                   Huber Regressor &     0.3262 &          -0.9212 \\
   2 &                Passive Aggressive &     0.0356 &          -0.2046 \\
   2 &            Ordinary Least Squares &    -7.0802 &       -4.022e+01 \\
   2 &                         Theil Sen & -1.069e+12 &       -3.650e+11 \\
   3 &       Stochastic Gradient Descent &     0.8352 &           0.6530 \\
   3 &                             Lasso &     0.8239 &           0.7205 \\
   3 &                       Elastic Net &     0.8232 &           0.6357 \\
   3 &                             Ridge &     0.6950 &           0.2380 \\
   3 &                               SVR &     0.5802 &          -1.1986 \\
   3 &                Passive Aggressive &    -1.5082 &          -3.5970 \\
   3 &                         Theil Sen & -3.826e+10 &       -5.744e+15 \\
   3 &            Ordinary Least Squares & -6.131e+10 &       -1.706e+16 \\
\bottomrule
\end{tabular}

In [66]:
t

,name,test_conf,deg,full_conf,unknown_conf,train_conf,gconf,cvSplit,ukSplit
0,SVR,0.239420,3,0.288537,0.237192,0.351966,0.292489,eager,sarek
1,Elastic Net,0.567548,1,0.633902,0.647746,0.657423,0.619173,chipseq,eager
2,Automatic Relevance Determination,0.770462,1,0.824186,0.806136,0.840844,0.811580,methylseq,chipseq
3,Huber Regressor,-0.216116,2,0.687638,0.636380,0.888270,0.378657,methylseq,chipseq
4,Ridge,0.840143,1,0.811758,0.786135,0.817479,0.823086,eager,chipseq
...,...,...,...,...,...,...,...,...,...
955,Elastic Net,0.770911,3,0.791131,0.727655,0.861099,0.807304,viralrecon,chipseq
956,Bayesian Ridge,0.793879,1,0.820010,0.849509,0.825182,0.812972,chipseq,eager
957,Ridge,0.787124,3,0.812154,0.775803,0.825051,0.808041,eager,chipseq
958,SVR,-0.084349,3,0.494113,0.064259,0.701049,0.328343,sarek,chipseq
